<a href="https://colab.research.google.com/github/shravan002/Top-up-Strategy/blob/master/Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import openpyxl as op
from google.colab import files
from openpyxl.utils.dataframe import dataframe_to_rows

In [ ]:
#create upload link
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
#copy json file in a directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

#permission to access kaggle data
!chmod 600 ~/.kaggle/kaggle.json

#Download kaggle data
!kaggle datasets download -d wendykan/lending-club-loan-data

#Unzip kaggle data
from zipfile import ZipFile
file_name = "lending-club-loan-data.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Unzip Done')

#Craete DataFrame with all data
data = pd.read_csv('loan.csv')
total_rows_Data=len(data.axes[0])
print('Total Rows in Data:', total_rows_Data)

 99% 711M/720M [00:26<00:00, 34.3MB/s]
100% 720M/720M [00:27<00:00, 27.9MB/s]
Unzip Done


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Total Rows in Data: 2260668


In [ ]:
#Create Working Dataframe
df = data.loc[:, ['annual_inc', 'loan_amnt', 'annual_inc_joint', 'issue_d', 'out_prncp', 'grade', 'loan_status', 'dti', 'dti_joint', 'acc_now_delinq', 'inq_last_12m', 'inq_fi', 'hardship_flag', 'collections_12_mths_ex_med']]
total_rows = len(df.axes[0])
print('Total Rows in DataFrame',total_rows)

#Replacing NaN value with 0
df['annual_inc'] = df['annual_inc'].replace(np.nan, 1)
df['annual_inc'] = df['annual_inc'].replace(0, 1)
df['annual_inc_joint'] = df['annual_inc_joint'].replace(np.nan, 0)
df['dti'] = df['dti'].replace(np.nan, 400000000)
df['dti_joint'] = df['dti_joint'].replace(np.nan, 0)
df['acc_now_delinq'] = df['acc_now_delinq'].replace(np.nan, 0)
df['inq_last_12m'] = df['inq_last_12m'].replace(np.nan, 1000)
df['inq_fi'] = df['inq_fi'].replace(np.nan, 1000)
df['collections_12_mths_ex_med'] = df['collections_12_mths_ex_med'].replace(np.nan, 0)

#Finding Total number of rows after data cleaning
total_rows = len(df.axes[0])
print('Total Rows in DataFrame',total_rows)

#Calculate Total income, dti and inq
sumColumn = df['annual_inc'] + df['annual_inc_joint']
df['total_inc'] = sumColumn

#Rounded real values to the nearest whole dollar
#df["total_inc"] = df["total_inc"].astype(int)

df['debt'] = df['dti'] * df['annual_inc']
df['debt_joint'] = df['dti_joint'] * df['annual_inc_joint']

df['total_dti'] = (df['debt'] + df['debt_joint']) / (df['annual_inc'] + df['annual_inc_joint'])

sumColumn2 = df['inq_last_12m'] + df['inq_fi']
df['total_inq'] = sumColumn2

#Generate Bands according to total_dti
dti_band = [-2, 10, 15, 20, 25, 100, 65000]
dti_bands_name =[' <=10', ' >10-15', ' >15-20', ' >20-25', ' >25', 'Missing']
df['dti_band'] = pd.cut(df['total_dti'], dti_band, labels = dti_bands_name)

#Generate Bands according to total_inquiry
inq_band = [-1, 0, 1, 2, 4, 100, 25000]
inq_bands_name =[' <=0', ' >0-1', ' >1-2', ' >2-4', ' >4', 'Missing']  
df['inq_band'] = pd.cut(df['total_inq'], inq_band, labels = inq_bands_name)

#Generate Band according to Total income
inc_bands = [-1, 1, 45000, 60000, 80000, 110000, 1100000000]
inc_bands_name =["Missing", "0-45K", ">45K-60K", ">60K-80K", ">80K-110K", ">110K"]
df['inc_band'] = pd.cut(df['total_inc'], inc_bands, labels = inc_bands_name)

#Create bad indicator
#IGP - Including Grace Period
#EGP - Excluding Grace Period
df['bad_flag_IGP'] = df['loan_status']
df['bad_flag_IGP'].replace(to_replace = ['Does not meet the credit policy. Status:Charged Off', 'Charged Off', 'In Grace Period',  'Default', 'Late (16-30 days)', 'Late (31-120 days)', 'Current', 'Fully Paid', 'Does not meet the credit policy. Status:Fully Paid'],
                                value = [1, 1, 1, 1, 1, 1, 0, 0, 0],
                              inplace = True)

df['bad_flag_EGP'] = df['loan_status']
df['bad_flag_EGP'].replace(to_replace = ['Does not meet the credit policy. Status:Charged Off', 'Charged Off', 'In Grace Period',  'Default', 'Late (16-30 days)', 'Late (31-120 days)', 'Current', 'Fully Paid', 'Does not meet the credit policy. Status:Fully Paid'],
                                value = [1, 1, 0, 1, 1, 1, 0, 0, 0],
                              inplace = True)

#Create buraeu bad indicator
df['hardship_flag_bin'] = df['hardship_flag']
df['hardship_flag_bin'].replace(to_replace = ['Y', 'N'],
                                value = [1, 0],
                              inplace = True)

df['delinq_flag'] = df['acc_now_delinq']
df['delinq_flag'].replace(to_replace = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                                value = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                              inplace = True)

df['collection_flag'] = df['collections_12_mths_ex_med']
df['collection_flag'].replace(to_replace = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                                value = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                              inplace = True)

df['bureau_bad'] = df['hardship_flag_bin'] + df['delinq_flag'] + df['collection_flag']

df['bureau_bad'].replace(to_replace = [0, 1, 2, 3],
                                value = [0, 1, 1, 1],
                              inplace = True)

#OA - Over All
df['OA_bad_acc_IGP'] = df['hardship_flag_bin'] + df['delinq_flag'] + df['collection_flag'] + df['bad_flag_IGP']

df['OA_bad_acc_IGP'].replace(to_replace = [0, 1, 2, 3, 4],
                            value = [0, 1, 1, 1, 1],
                              inplace = True)

#OA - Over All
df['OA_bad_acc_EGP'] = df['hardship_flag_bin'] + df['delinq_flag'] + df['collection_flag'] + df['bad_flag_EGP']

df['OA_bad_acc_EGP'].replace(to_replace = [0, 1, 2, 3, 4],
                            value = [0, 1, 1, 1, 1],
                              inplace = True)

#Temp column to calculate the total no of accounts in different segment
df['total_acc'] = df['hardship_flag']
df['total_acc'].replace(to_replace = ['Y', 'N'],
                        value = [1, 1],
                        inplace = True)

df.tail()

Total Rows in DataFrame 2260668
Total Rows in DataFrame 2260668


,annual_inc,loan_amnt,annual_inc_joint,issue_d,out_prncp,grade,loan_status,dti,dti_joint,acc_now_delinq,inq_last_12m,inq_fi,hardship_flag,collections_12_mths_ex_med,total_inc,debt,debt_joint,total_dti,total_inq,dti_band,inq_band,inc_band,bad_flag_IGP,bad_flag_EGP,hardship_flag_bin,delinq_flag,collection_flag,bureau_bad,OA_bad_acc_IGP,OA_bad_acc_EGP,total_acc
2260663,58000.0,12000,0.0,Oct-2017,8687.20,C,Current,20.88,0.00,0.0,1.0,2.0,N,0.0,58000.0,1211040.0,0.0,20.880000,3.0,>20-25,>2-4,>45K-60K,0,0,0,0.0,0.0,0.0,0.0,0.0,1
2260664,30000.0,12000,135000.0,Oct-2017,0.00,E,Fully Paid,19.28,7.36,0.0,7.0,5.0,N,0.0,165000.0,578400.0,993600.0,9.527273,12.0,<=10,>4,>110K,0,0,0,0.0,0.0,0.0,0.0,0.0,1
2260665,64000.0,10000,0.0,Oct-2017,5993.27,B,Current,12.96,0.00,0.0,3.0,0.0,N,1.0,64000.0,829440.0,0.0,12.960000,3.0,>10-15,>2-4,>60K-80K,0,0,0,0.0,1.0,1.0,1.0,1.0,1
2260666,60000.0,12000,0.0,Oct-2017,9924.69,D,Current,30.82,0.00,0.0,4.0,0.0,N,0.0,60000.0,1849200.0,0.0,30.820000,4.0,>25,>2-4,>45K-60K,0,0,0,0.0,0.0,0.0,0.0,0.0,1
2260667,60000.0,16550,0.0,Oct-2017,13541.01,D,Current,18.40,0.00,0.0,2.0,0.0,N,0.0,60000.0,1104000.0,0.0,18.400000,2.0,>15-20,>1-2,>45K-60K,0,0,0,0.0,0.0,0.0,0.0,0.0,1


**Cartesion Product of different demographics and volume by different KPI (Key point indicators)**

In [ ]:
#indicator = 'OA_bad_acc_IGP'
#indicator = 'OA_bad_acc_EGP'
indicator = 'temp'
#indicator = 'loan_status'
#indicator = 'bureau_bad'
x = df.groupby(
   ['dti_band', 'inq_band', 'inc_band', 'grade', indicator]
).agg( 
    {
          indicator: "count",
    }
)

x = x.unstack()

filename = "temp.csv"
x.to_csv(filename)
df1 = pd.read_csv('temp.csv', low_memory=False)

df1.tail()

In [ ]:
dft = pd.DataFrame()
dft= df['annual_inc']
dft.head()

0    55000.0
1    90000.0
2    59280.0
3    92000.0
4    57250.0
Name: annual_inc, dtype: float64

In [ ]:
#create upload link
uploaded = files.upload()

Saving Segments.csv to Segments.csv


In [ ]:
#Create DataFrame with all data
strategy_data = pd.read_csv('Segments.csv')
df1 = strategy_data

#Finding Total number of rows 
total_rows = len(df1.axes[0])
print('Total Rows in DataFrame',total_rows)
df1.head()

Total Rows in DataFrame 1512


,dti_band,inq_band,inc_band,grade,Total Account,OA_bad_acc_IGP,OA_bad_acc_EGP
0,<=10,<=0,Missing,A,0,0,0
1,<=10,<=0,Missing,B,0,0,0
2,<=10,<=0,Missing,C,0,0,0
3,<=10,<=0,Missing,D,0,0,0
4,<=10,<=0,Missing,E,0,0,0


In [ ]:
df1['OA_bad_rate_IGP'] = df1['OA_bad_acc_IGP'] / df1['Total Account'] * 100
df1['OA_bad_rate_EGP'] = df1['OA_bad_acc_EGP'] / df1['Total Account'] * 100

df1['OA_bad_rate_IGP'] = df1['OA_bad_rate_IGP'].replace(np.nan, 0)
df1['OA_bad_rate_EGP'] = df1['OA_bad_rate_EGP'].replace(np.nan, 0)

total_acc = df1['Total Account'].sum(axis = 0, skipna = True) 
OA_bad_IGP_acc = df1['OA_bad_acc_IGP'].sum(axis = 0, skipna = True) 
OA_bad_EGP_acc = df1['OA_bad_acc_EGP'].sum(axis = 0, skipna = True) 

avg_OA_bad_rate_IGP = OA_bad_IGP_acc / total_acc *100
avg_OA_bad_rate_EGP = OA_bad_EGP_acc / total_acc *100
print(avg_OA_bad_rate_IGP)
print(avg_OA_bad_rate_EGP)

#Finding Total number of rows
total_rows = len(df1.axes[0])
print('Total Segments',total_rows)

#Removing Segments with Zero Accounts
df1.drop(df1[df1['Total Account'] == 0].index, inplace=True)
 
total_rows = len(df1.axes[0])
print('Removed segment with zero acc',total_rows)

#Removing Segments which is greater than Over all bad rate
df1.drop(df1[df1['OA_bad_rate_EGP'] > avg_OA_bad_rate_EGP].index, inplace=True)

total_rows = len(df1.axes[0])
print('Removed segment which is more than avg bad rate',total_rows)

#Removing segments with Missing Value in DTI and Income
df1.drop(df1[df1['dti_band'] == 'Missing'].index, inplace=True)
df1.drop(df1[df1['inc_band'] == 'Missing'].index, inplace=True)

total_rows = len(df1.axes[0])
print('Removed Segments with Missing Values of DTI and Income',total_rows)

#Removing Segments with DTI greater than 20
df1.drop(df1[df1['dti_band'] == ' >20-25'].index, inplace=True)
df1.drop(df1[df1['dti_band'] == ' >25'].index, inplace=True)

total_rows = len(df1.axes[0])
print('Removed Segments which have DTI greater than 20:',total_rows)

#Removing Segments with Inquiry greater than 4
df1.drop(df1[df1['inq_band'] == ' >4'].index, inplace=True)

total_rows = len(df1.axes[0])
print('Removed Segments which have Inquiry Greater than 4:',total_rows)

#Removing Segments with Income less than 80K
df1.drop(df1[df1['inc_band'] == '0-45K'].index, inplace=True)
df1.drop(df1[df1['inc_band'] == '>45K-60K'].index, inplace=True)
df1.drop(df1[df1['inc_band'] == '>60K-80K'].index, inplace=True)

total_rows = len(df1.axes[0])
print('Removed Segments which have Income less than 80K:',total_rows)

#Removing Segments with Grade C, D, E, F, G
df1.drop(df1[df1['grade'] == 'C'].index, inplace=True)
df1.drop(df1[df1['grade'] == 'D'].index, inplace=True)
df1.drop(df1[df1['grade'] == 'E'].index, inplace=True)
df1.drop(df1[df1['grade'] == 'F'].index, inplace=True)
df1.drop(df1[df1['grade'] == 'G'].index, inplace=True)

total_rows = len(df1.axes[0])
print('Removed Segments which have Grade C, D, E, F, G:',total_rows)

#df1 = df1.sort_values(by=['OA_bad_rate_IGP'], ascending=True)
filename = "cut-off.csv"
df1.to_csv(filename)
df2 = pd.read_csv('cut-off.csv', low_memory=False)
df2.tail()

14.839286440998855
14.453471274862121
Total Segments 1512
Removed segment with zero acc 1200
Removed segment which is more than avg bad rate 523
Removed Segments with Missing Values of DTI and Income 404
Removed Segments which have DTI greater than 20: 246
Removed Segments which have Inquiry Greater than 4: 211
Removed Segments which have Income less than 80K: 92
Removed Segments which have Grade C, D, E, F, G: 60


,Unnamed: 0,dti_band,inq_band,inc_band,grade,Total Account,OA_bad_acc_IGP,OA_bad_acc_EGP,OA_bad_rate_IGP,OA_bad_rate_EGP
55,666,>15-20,>2-4,>110K,B,4592,365,341,7.948606,7.425958
56,742,>15-20,Missing,>80K-110K,A,7605,418,418,5.496384,5.496384
57,743,>15-20,Missing,>80K-110K,B,10723,1303,1297,12.151450,12.095496
58,749,>15-20,Missing,>110K,A,5869,307,307,5.230874,5.230874
59,750,>15-20,Missing,>110K,B,7129,821,816,11.516342,11.446206


In [ ]:
#df1.drop(df1[df1['OA_bad_rate_IGP'] >= 10].index, inplace=True)

# lowest5_prcnt_acc = int(5 / 100 * total_rows)
# df1.drop(df1.tail(lowest5_prcnt_acc).index,inplace=True)

# #Finding Total number of rows after removing the segments greater than avg bad rate
# total_rows = len(df1.axes[0])
# print('Removed segment last 5%',total_rows) 

In [ ]:
values, count = np.unique(df['loan_status'], return_counts=True)

dfTemp = pd.DataFrame()
dfTemp['Values'] = values
dfTemp['Count'] = count

dfTemp.head()
# filename = "temp.csv"
# dfTemp.to_csv(filename)
# data1 = pd.read_csv('temp.csv', low_memory=False)